In [76]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [77]:
standing_url = "https://sofifa.com/"

In [79]:
links_data = requests.get(standing_url)

In [80]:
links_soup = BeautifulSoup(links_data.text)

In [81]:
select_tbody = links_soup.select("body > div.center > div > div.col.col-12 > div > table > tbody")[0]

In [82]:
athlete_links_a_unselected = select_tbody.find_all("a")

In [83]:
# 利用if 选择运动员link，获得60个运动员的数据链接
athlete_links = ["https://sofifa.com" + link_a.get("href") for link_a in athlete_links_a_unselected if link_a.get("data-tip-pos") == "top"]

In [84]:
athlete_link = athlete_links[0]
athlete_link

'https://sofifa.com/player/268421/mathys-tel/230005/'

开始爬取运动员页面，例如，https://sofifa.com/player/268421/mathys-tel/230005/

In [85]:
athlete_data = requests.get(athlete_link)

In [152]:
athlete_soup = BeautifulSoup(athlete_data.text)
# overall_rating, potential, value, wage
name_list = ["overall_rating", "potential", "value", "wage"]
data_list = list(range(len(name_list)))

In [158]:
# overall rating 
overall_rating = athlete_soup.select("#body > div:nth-child(5) > div > div.col.col-12 > div.bp3-card.player > section > div:nth-child(1) > div > span")
overall_rating_data = overall_rating[0].string
data_list[0] = int(overall_rating_data)

# potential
potential = athlete_soup.select("#body > div:nth-child(5) > div > div.col.col-12 > div.bp3-card.player > section > div:nth-child(2) > div > span")
potential_data = potential[0].string
potential_data
data_list[1] = int(potential_data)

#Value
value = athlete_soup.select("#body > div:nth-child(5) > div > div.col.col-12 > div.bp3-card.player > section > div:nth-child(3) > div")
value_data = str(value[0])
value_re = re.findall("(\d+\.?\d*)([MK])", value_data)
value_num = value_re[0][0]
value_unit = value_re[0][1]
if value_unit == "M":
    data_list[2] = (float(value_num)*10**6)
elif value_unit == "K":
    data_list[2] = (float(value_num*10**3))
else:
    print("clawer value failed.")

# Wage
wage = athlete_soup.select("#body > div:nth-child(5) > div > div.col.col-12 > div.bp3-card.player > section > div:nth-child(4)")
wage_data = str(wage[0])
wage_re = re.findall("(\d+\.?\d*)([MK])", wage_data)
wage_num = wage_re[0][0]
wage_unit = wage_re[0][1]
if wage_unit == "M":
    data_list[3] = (float(wage_num)*10**6)
elif wage_unit == "K":
    data_list[3] = (float(wage_num)*10**3)
else:
    print("clawer value failed.")

2


In [162]:
print(data_list)
print(name_list)

[64, 84, 1600000.0, 2000.0]
['overall_rating', 'potential', 'value', 'wage']
